In [ ]:
!pip install yfinance
!pip install ta

In [ ]:
import pandas as pd
import datetime as dt
from pandas_datareader import data as pdr
import yfinance as yfin
import time
import os

In [ ]:
yfin.pdr_override()

In [ ]:
symbols = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
symbols = symbols.Symbol.to_list()
symbols

In [ ]:
path = (r"/content/drive/MyDrive/Colab Notebooks/Screener/")

In [ ]:
#import shutil
#shutil.rmtree('/content/drive/MyDrive/Colab Notebooks/csvs')

startyear = 2021
startmonth = 1
startday = 1
start = dt.datetime(startyear, startmonth, startday)
now = pd.Timestamp("now")
count = 0

for stock in symbols:
  try:
    count += 1
    df = pdr.get_data_yahoo(stock, start, now)
    print(f'Checking {count}.....{stock}')
    df.to_csv(path + stock + '.csv')
  except Exception as e:
    print("no results found on "+stock)
    print(e)
    pass

In [ ]:
def MACD_signal_up(df):
  from ta.trend import MACD
  indicator_macd = MACD(df['Adj Close'])
  df['MACD'] = indicator_macd.macd()
  df['Signal'] = indicator_macd.macd.signal()
  df['MACD Histogram'] = indicator_macd.macd_diff()
  df['Below_0_Crossover_MACD_Signal'] = False
  df['Simple_Crossover_MACD_Signal'] = False

  #Define the logics:
  if (df[-2:]['MACD'].values[0] <= df[-1:]['MACD'].values[0]) and (df[-2:]['MACD'].values[0] <= df[-2:]['Signal'].values[0]) and (df[-1:]['MACD'].values[0] >= df[-1:]['Signal'].values[0]):

    ## MACD crossover AND Below 0
    if (df[-2:]['MACD'].values[0] <= df[-1:]['MACD'].values[0]) and (df[-2:]['MACD'].values[0] <= df[-2:]['Signal'].values[0]) and (df[-1:]['MACD'].values[0] >= df[-1:]['Signal'].values[0]) and df[-1:]['MACD'].values[0] <= 0:
      print(f'{symbol} MACD UP alert below 0')
      df['Below_0_Crossover_MACD_Signal'][-1] = True

    else:
      print(f'{symbol} MACD UP alert')
      df['Simple_Crossover_MACD_Signal'][-1] = True
      df['Below_0_Crossover_MACD_Signal'][-1] = False
      return True
  
  return False
  
def MACD_signal_down(df):
  from ta.trend import MACD
  indicator_macd = MACD(df['Adj Close'])
  df['MACD'] = indicator_macd.macd()
  df['Signal'] = indicator_macd.macd.signal()
  df['MACD Histogram'] = indicator_macd.macd_diff()
  df['Simple_Crossover_MACD_Signal'] = False
  df['Above_0_Crossunder_MACD_Signal'] = False
  
  if(df[-2:]['MACD'].values[0] >= df[-1:]['MACD'].values[0]) and (df[-2:]['MACD'].values[0] >= df[-2:]['Signal'].values[0]) and (df[-1:]['MACD'].values[0] <= df[-1:]['Signal'].values[0]):
    ##MACD crossover AND above 0
    if(df[-2:]['MACD'].values[0] >= df[-1:]['MACD'].values[0]) and (df[-2:]['MACD'].values[0] >= df[-2:]['Signal'].values[0]) and (df[-1:]['MACD'].values[0] <= df[-1:]['Signal'].values[0]) and df[-1:]['MACD'].values[0] >= 0:
      print(f'{symbol} MACD DOWN above 0')
      df['Above_0_Crossunder_MACD_Signal'][-1] = True
    
    else:
      print(f'{symbol} MACD DOWN alert')
      df['Simple_Crossover_MACD_Signal'] = True
      df['Above_0_Crossunder_MACD_Signal'] = False
      return True
    return False

def Bollinger_signal_up(df, window=20, window_dev=2):
  from ta.volatility import BollingerBands
  indicator_bb = BollingerBands(df['Adj Close'], 20, 2)
  df['bb_bbm'] = indicator_bb.bollinger_mavg()
  df['bb_bbh'] = indicator_bb.bollinger_hband()
  df['bb_bbl'] = indicator_bb.bollinger_lband()
  df['bb_bbhi'] = indicator_bb.hband_indicator()
  df['bb_bbli'] = indicator_bb.lband_indicator()
  df['Boll_UP'] = False
  df['Boll_UP2'] = False

  if (df[-2:]['bb_bbhi'].values[0] == 0) and (df[-1:]['bb_bbhi'].values[0] == 1):
    df['Boll_UP'][-1] = True
    print(f'{symbol} Went above HIGH bollinger JUST NOW')
    return True
  elif (df[-2:]['bb_bbhi'].values[0] == 1) and (df[-1:]['bb_bbhi'].values[0] == 1):
    df['Boll_UP2'][-1] = True
    print(f'{symbol} is ALREADY on HIGH bollinger')
    return True
  return False

def Bollinger_signal_down(df, window=20, window_dev=2):
  from ta.volatility import BollingerBands
  indicator_bb = BollingerBands(df["Adj Close"], 20, 2)
  df['bb_bbm'] = indicator_bb.bollinger_mavg()
  df['bb_bbh'] = indicator_bb.bollinger_hband()
  df['bb_bbl'] = indicator_bb.bollinger_lband()
  df['bb_bbhi'] = indicator_bb.hband_indicator()
  df['bb_bbli'] = indicator_bb.lband_indicator()
  df['Boll_DOWN'] = False
  df['Boll_DOWN2'] = False

  if (df[-2:]['bb_bbli'].values[0] == 0) and (df[-1:]['bb_bbli'].values[0] == 1):
    print(f'{symbol} is below LOWER bollinger JUST NOW')
    df['BOLL_Down'][-1] = True
    return True
  elif (df[-2:]['bb_bbli'].values[0] == 1) and (df[-1:]['bb_bbli'].values[0] == 1):
    df['Boll_Down2'][-1] = True
    print(f'{symbol} is ALREADY on LOWER bollinger')
    return True
  return False

def RSI_signal_up(df, window = 14):
  from ta.momentum import RSIIndicator
  indicator_rsi = RSIIndicator(df["Adj Close"], window = 14)
  df['RSI'] = indicator_rsi.rsi()
  df['RSI_Overbought'] = False

  if (df[-2:]['RSI'].values[0] >= 70) and (df[-1:]['RSI'].values[0] <= 70):
    print(f'{symbol} RSI ALERT just went to range from overbought')

  if (df[-1:]['RSI'].values[0] >= 70):

    print(f'{symbol} is overbought')
    df['RSI_Overbought'][-1] = True

    if (df[-2:]['RSI'].values[0] <= 70) and (df[-1:]['RSI'].values[0] >= 70):
      print(f'{symbol} RSI ALERT just crossed over')
      return True
  return False

def RSI_signal_down(df, window = 14):
  from ta.momentum import RSIIndicator
  indicator_rsi = RSIIndicator(df["Adj Close"], window = 14)
  df['RSI'] = indicator_rsi.rsi()
  df['RSI_Oversold'] = False

  if (df[-2:]['RSI'].values[0] <= 30) and (df[-1:]['RSI'].values[0] >= 30):
    print(f'{symbol} RSI ALERT just went to range from oversold')

  if (df[-1:]['RSI'].values[0] <= 30):

    print(f'{symbol} is oversold')
    df['RSI_Oversold'][-1] = True

    if (df[-2:]['RSI'].values[0] >= 30) and (df[-1:]['RSI'].values[0] <= 30):
      print(f'{symbol} RSI ALERT just crossed under')
      return True
  return False

# Price action Functions (candlestick patterns, consolidations, breakouts etc)

def consolidating_signal(df, perc = 3.5):
  
  range_of_candlesticks = df[-15:]

  max_close_price = range_of_candlesticks['Adj Close'].max()
  min_close_price = range_of_candlesticks['Adj Close'].min()
  threshold_detection = 1 - (perc / 100)

  if min_close_price > (max_close_price * threshold_detection):
    return True
  return False

def breaking_out_signal(df, perc=1,):
  last_close = df[-1:]['Adj Close'].values[0]
  if consolidating_signal(df[-1:], perc=perc):
    recent_close = df[-16:-1]
    if last_close > recent_close['Adj Close'].max():
      return True
  return False

In [ ]:
print('--------------------- START SCREENER ----------------------------')

print('--------------------- BREAKOUTS ----------------------------')
for filename in os.listdir(path):
  df = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/Screener/{}".format(filename))
  symbol = filename.split(".")[0]

  if breaking_out_signal(df, 3):
    print(f'{symbol} is breaking out')

print('--------------------- STILL IN CONSOLIDATION ----------------------------') 
for filename in os.listdir(path):
  df = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/Screener/{}".format(filename))
  symbol = filename.split(".")[0]

  if consolidating_signal(df, perc=2):
    print(f'{symbol} is consolidating')

print('--------------------- RSI ----------------------------') 
for filename in os.listdir(path):
  df = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/Screener/{}".format(filename))
  symbol = filename.split(".")[0]

  if RSI_signal_up(df):
    print(f'{symbol} is on RSI alert UP')
  if RSI_signal_down(df):
    print(f'{symbol} is on RSI alert DOWN')

print('--------------------- MACD ----------------------------') 
for filename in os.listdir(path):
  df = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/Screener/{}".format(filename))
  symbol = filename.split(".")[0]

  if MACD_signal_up(df):
    print(f'{symbol} is on MACD UP')
  if MACD_signal_down(df):
    print(f'{symbol} is on MACD DOWN')

print('--------------------- BOLLINGER ----------------------------') 
for filename in os.listdir(path):
  df = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/Screener/{}".format(filename))
  symbol = filename.split(".")[0]

  if Bollinger_signal_up(df):
    print(f'{symbol} is on BOLLINGER UP')
  if Bollinger_signal_down(df):
    print(f'{symbol} is on BOLLINGER DOWN')

--------------------- START SCREENER ----------------------------
